In [1]:
from methods import *
import os, shutil

In [4]:
#user inputs

#load hyperparameters
#sizes = ['1_tiny', '2_small', '3_standard', '4_full']
sizes = ['2_small', '3_standard', '4_full']
size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]

alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [5]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [6]:
def run_rnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_model(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [8]:
#for each method

writer_cnn = open('outputs_f1/cnn_general'+'_' + get_now_str() + '.txt', 'w')
writer_rnn = open('outputs_f1/rnn_general' + '_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    writer_rnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_rnn = {alpha:[] for alpha in alphas}
    performances_cnn = {alpha:[] for alpha in alphas}

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        #test each alpha value
        for alpha in alphas:

            train_path = dataset_folder + '/train_weda_' + str(alpha) + '.txt'
            test_path = 'size_data_f1/test/' + dataset + '/test.txt'
            acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
            print("cnn aug ", acc, alpha, a_method, dataset_folder)

            performances_cnn[alpha].append(acc)
            """
            acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
            performances_rnn[alpha].append(acc)
            print("rnn aug", acc)
            """

    writer_cnn.write(str(performances_cnn) + '\n')
    writer_rnn.write(str(performances_rnn)+"\n")
    for alpha in performances_cnn:
        line = str(alpha) + ' : ' + str(sum(performances_cnn[alpha])/len(performances_cnn[alpha]))
        writer_cnn.write(line + '\n')
        print(line)
    """
    for alpha in performances_rnn:
        line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
        writer_rnn.write(line + '\n')
        print(line)
    """
    print(performances_cnn)
    print(performances_rnn)

writer_cnn.close()
writer_rnn.close()
    
## TODO: add training without augmentation

Epoch 1/20
59/59 [==============================] - 13s 207ms/step - loss: 0.5650 - accuracy: 0.7016 - val_loss: 0.2695 - val_accuracy: 0.8991
Epoch 2/20
59/59 [==============================] - 10s 176ms/step - loss: 0.2038 - accuracy: 0.9361 - val_loss: 0.0957 - val_accuracy: 0.9752
Epoch 3/20
59/59 [==============================] - 10s 166ms/step - loss: 0.0586 - accuracy: 0.9894 - val_loss: 0.0420 - val_accuracy: 0.9888
Epoch 4/20
59/59 [==============================] - 10s 174ms/step - loss: 0.0187 - accuracy: 0.9983 - val_loss: 0.0253 - val_accuracy: 0.9933
Epoch 5/20
59/59 [==============================] - 12s 197ms/step - loss: 0.0084 - accuracy: 0.9994 - val_loss: 0.0188 - val_accuracy: 0.9948
Epoch 6/20
59/59 [==============================] - 11s 190ms/step - loss: 0.0045 - accuracy: 0.9995 - val_loss: 0.0157 - val_accuracy: 0.9955
Epoch 7/20
59/59 [==============================] - 11s 180ms/step - loss: 0.0027 - accuracy: 0.9999 - val_loss: 0.0138 - val_accuracy: 0.9964

NameError: name 'a_method' is not defined

In [ ]:

writer_cnn = open('outputs_f1/cnn_no_aug_' + get_now_str() + '.txt', 'w')
writer_rnn = open('outputs_f1/rnn_no_aug_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    writer_rnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_rnn = []
    performances_cnn = []

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        train_path = dataset_folder + '/train_orig.txt'
        test_path = 'size_data_f1/test/' + dataset + '/test.txt'
        
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn", acc)
        performances_cnn.append(acc)

        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn.append(acc)
        print("rnn", acc)
        """

    writer_cnn.write(str(performances_cnn) + '\n')
    writer_rnn.write(str(performances_rnn)+"\n")

    print(performances_cnn)
    print(performances_rnn)

writer_cnn.close()
writer_rnn.close()

